In [22]:
import pandas, numpy

## functions & options


In [23]:
input_file_directory = '/Users/kja11/OneDrive - Háskóli Íslands/PhD ATG7//0 in_silico/Python/1)data_input/'

In [24]:
output_file_directory = '/Users/kja11/OneDrive - Háskóli Íslands/PhD ATG7/0 in_silico/Python/3)output/'

# ATG7 expression, ensembl. Data preparation 

In [25]:
%%time
path = input_file_directory + "xenabrowser_brut_data/xena_surv_ATG7.tsv"
df_ori= pandas.read_csv(path, sep = "\t")
print(df_ori.shape)
df_ori.head()

(19131, 31)
Wall time: 93.8 ms


,sample,samples,_sample_type,_primary_site,ENSG00000197548.12,ENST00000451513.5,ENST00000435760.5,ENST00000451830.5,ENST00000460444.5,ENST00000470474.1,...,ENST00000434066.6,ENST00000478638.5,ENST00000461278.1,ENST00000460291.1,ENST00000467121.1,ENST00000414717.5,ENST00000427759.5,ENST00000446110.1,OS,OS.time
0,TARGET-20-PARUBT-40,TARGET-20-PARUBT-40,Recurrent Blood Derived Cancer - Peripheral Blood,White blood cell,4.401,-9.9660,0.3685,2.656,-0.7108,-1.0860,...,-9.9660,-9.966,-0.6193,-5.0120,-9.966,0.8568,-9.9660,-9.9660,NaN,NaN
1,TARGET-20-PATJHJ-40,TARGET-20-PATJHJ-40,Recurrent Blood Derived Cancer - Peripheral Blood,White blood cell,2.820,-9.9660,-0.1345,-1.595,-2.9320,-9.9660,...,-9.9660,-9.966,-9.9660,-3.3080,-9.966,-2.0530,-2.7270,-0.2328,NaN,NaN
2,TARGET-10-PASLZM-40,TARGET-10-PASLZM-40,Recurrent Blood Derived Cancer - Peripheral Blood,White blood cell,1.158,-9.9660,-2.6350,-1.994,-9.9660,-9.9660,...,-9.9660,-9.966,-9.9660,-3.6260,-9.966,-1.0860,-9.9660,-9.9660,NaN,NaN
3,TARGET-21-PATAIJ-42,TARGET-21-PATAIJ-42,Post treatment Blood Cancer - Blood,White blood cell,4.332,0.2642,1.0850,-1.086,-9.9660,-9.9660,...,-9.9660,-9.966,-9.9660,-1.5950,-9.966,0.3115,-0.6643,1.7230,NaN,NaN
4,TARGET-21-PASVJS-41,TARGET-21-PASVJS-41,Post treatment Blood Cancer - Bone Marrow,White blood cell,5.156,-9.9660,1.6280,-1.732,-2.8260,-0.6643,...,-0.1187,-9.966,-1.3550,0.4125,-9.966,0.2522,-9.9660,2.4310,NaN,NaN


In [26]:
# Column selection
df = df_ori[['sample',
            '_sample_type',
            '_primary_site',
             'OS', 'OS.time',
            'ENSG00000197548.12',
            'ENST00000354449.7', 
            'ENST00000354956.9',
            'ENST00000446450.6']]

# We have Adrenal Gland and Adrenal gland. I need to put "G".
df = df.assign(_primary_site=df['_primary_site'].str.title())

# Column rename
df = df.rename(columns = {'sample' : 'sample',
                    '_sample_type' : 'Sample_Type',
                    'OS.time' : 'OS_time',
                    '_primary_site' : 'Primary_Site',
                    'ENSG00000197548.12': 'ATG7',
                    'ENST00000354449.7': 'ATG7_1',
                    'ENST00000354956.9': 'ATG7_2',
                    'ENST00000446450.6': 'ATG7_3'})

# Calcul to have value from log2;  2**(ATG7(1))-0.001
df = df.assign(ATG7_total= pow(2,df['ATG7'])-0.001,
          ATG7_1tpm= pow(2,df['ATG7_1'])-0.001,
          ATG7_2tpm= pow(2,df['ATG7_2'])-0.001,
          ATG7_3tpm= pow(2,df['ATG7_3'])-0.001)

# Calcul to have log2+1 from value; 
df = df.assign(log2_p1_ATG7_total= numpy.log2(df['ATG7_total']+1),
               log2_p1_ATG7_1= numpy.log2(df['ATG7_1tpm']+1),
               log2_p1_ATG7_2= numpy.log2(df['ATG7_2tpm']+1))

# Calcul to see percentage of expression of the 3 isoforms of ATG7 
df = df.assign(ATG7_prot_tot= df['ATG7_1tpm'] + df['ATG7_2tpm'] + df['ATG7_3tpm'])

df = df.assign(ATG7_1perc= (df['ATG7_1tpm'] *100)/ df['ATG7_prot_tot'],
              ATG7_2perc= (df['ATG7_2tpm'] *100)/ df['ATG7_prot_tot'],
              ATG7_3perc= (df['ATG7_3tpm'] *100)/ df['ATG7_prot_tot'])                              

#Add columns, normalized on total
df = df.assign(ATG7_1norm= df['ATG7_1tpm']/df["ATG7_total"],
               ATG7_2norm= df['ATG7_2tpm']/df["ATG7_total"])                                   

df.head()

,sample,Sample_Type,Primary_Site,OS,OS_time,ATG7,ATG7_1,ATG7_2,ATG7_3,ATG7_total,...,ATG7_3tpm,log2_p1_ATG7_total,log2_p1_ATG7_1,log2_p1_ATG7_2,ATG7_prot_tot,ATG7_1perc,ATG7_2perc,ATG7_3perc,ATG7_1norm,ATG7_2norm
0,TARGET-20-PARUBT-40,Recurrent Blood Derived Cancer - Peripheral Blood,White Blood Cell,NaN,NaN,4.401,1.3900,1.522,-1.7320,21.125765,...,3.000343e-01,4.467655,1.855905,1.952665,5.790710,45.241201,49.577495,5.181305,0.124009,0.135895
1,TARGET-20-PATJHJ-40,Recurrent Blood Derived Cancer - Peripheral Blood,White Blood Cell,NaN,NaN,2.820,1.1180,-1.086,-9.9660,7.060624,...,-1.495113e-07,3.010892,1.664236,0.555881,2.639524,82.191280,17.808726,-0.000006,0.307262,0.066576
2,TARGET-10-PASLZM-40,Recurrent Blood Derived Cancer - Peripheral Blood,White Blood Cell,NaN,NaN,1.158,0.0158,-3.171,-3.8160,2.230479,...,7.000183e-02,1.691748,1.007204,0.150597,1.190042,84.871949,9.245753,5.882299,0.452823,0.049329
3,TARGET-21-PATAIJ-42,Post treatment Blood Cancer - Blood,White Blood Cell,NaN,NaN,4.332,1.3450,1.975,-9.9660,20.139115,...,-1.495113e-07,4.401843,1.823465,2.301670,6.469584,39.249848,60.750154,-0.000002,0.126088,0.195157
4,TARGET-21-PASVJS-41,Post treatment Blood Cancer - Bone Marrow,White Blood Cell,NaN,NaN,5.156,1.8040,3.769,0.5069,35.653197,...,1.419994e+00,5.195867,2.166995,3.871026,18.542569,18.826249,73.515731,7.658019,0.097912,0.382342


In [27]:
#what kind of samples are the data ? 
df['Sample_Type'].value_counts()

Primary Tumor                                        9185
Normal Tissue                                        7429
Solid Tissue Normal                                   738
Cell Line                                             433
Metastatic                                            393
Primary Solid Tumor                                   286
Primary Blood Derived Cancer - Peripheral Blood       239
Primary Blood Derived Cancer - Bone Marrow            237
Recurrent Blood Derived Cancer - Bone Marrow          104
Recurrent Tumor                                        45
Recurrent Solid Tumor                                  13
Post treatment Blood Cancer - Bone Marrow              12
Additional - New Primary                               11
Recurrent Blood Derived Cancer - Peripheral Blood       3
Additional Metastatic                                   1
Post treatment Blood Cancer - Blood                     1
Control Analyte                                         1
Name: Sample_T

In [28]:
#How many each organs are the data ? 
df['Primary_Site'].value_counts()

Brain                         1846
Lung                          1410
Breast                        1391
Skin                          1282
Kidney                        1193
Esophagus                      848
Prostate                       648
Colon                          639
Stomach                        624
Blood Vessel                   606
White Blood Cell               595
Thyroid Gland                  571
Head And Neck Region           564
Liver                          531
Ovary                          515
Adipose Tissue                 515
Blood                          444
Bladder                        435
Muscle                         396
Heart                          377
Pancreas                       350
Testis                         319
Cervix                         309
Thyroid                        279
Nerve                          278
Soft Tissue,Bone               264
Adrenal Gland                  205
Endometrium                    204
Paraganglia         

In [29]:
#Creation of two dataframe, Normal tissue and Primary tumor.
df_ATG7_Normal = df[df['Sample_Type'].str.match('Normal Tissue')]
df_ATG7_Primary = df[df['Sample_Type'].str.match('(Primary Tumor)')]

print('normal_tissue' , len(df_ATG7_Normal))
print('primary_tumors' , len(df_ATG7_Primary))

normal_tissue 7429
primary_tumors 9185


In [31]:
# create df with only Normal and Primary 
df_ATG7_NormPrim = pandas.concat([df_ATG7_Normal, df_ATG7_Primary], axis=0)
df_ATG7_NormPrim.reset_index(drop=True, inplace=True)
df_ATG7_NormPrim.head()

,sample,Sample_Type,Primary_Site,OS,OS_time,ATG7,ATG7_1,ATG7_2,ATG7_3,ATG7_total,...,ATG7_3tpm,log2_p1_ATG7_total,log2_p1_ATG7_1,log2_p1_ATG7_2,ATG7_prot_tot,ATG7_1perc,ATG7_2perc,ATG7_3perc,ATG7_1norm,ATG7_2norm
0,GTEX-ZTTD-0326-SM-57WFW,Normal Tissue,Muscle,NaN,NaN,5.042,3.860,2.390,0.2642,32.944283,...,1.199970e+00,5.085097,3.955992,2.641679,20.959850,69.271996,25.002916,5.725088,0.440723,0.159074
1,GTEX-PX3G-1626-SM-2S1PT,Normal Tissue,Muscle,NaN,NaN,4.252,2.322,1.956,1.1320,19.052710,...,2.190624e+00,4.325725,2.584782,2.286540,11.068720,45.165557,35.043325,19.791118,0.262390,0.203585
2,GTEX-OXRO-1726-SM-3LK6C,Normal Tissue,Muscle,NaN,NaN,3.909,2.602,1.345,-9.9660,15.020948,...,-1.495113e-07,4.001888,2.821767,1.823465,8.609579,70.506086,29.493916,-0.000002,0.404121,0.169051
3,GTEX-13JVG-0126-SM-5L3D3,Normal Tissue,Muscle,NaN,NaN,3.854,2.568,1.428,-9.9660,14.459044,...,-1.495113e-07,3.950379,2.792620,1.883517,8.618602,68.791526,31.208476,-0.000002,0.410046,0.186024
4,GTEX-NPJ8-1626-SM-2HMIY,Normal Tissue,Muscle,NaN,NaN,3.822,2.164,1.293,-1.5100,14.141840,...,3.501112e-01,3.920469,2.454322,1.786333,7.280039,61.545783,33.645022,4.809194,0.316830,0.173200


In [32]:
df_ATG7_NormPrim['Primary_Site'].value_counts()

Brain                      1814
Lung                       1299
Breast                     1271
Kidney                      912
Esophagus                   834
Skin                        658
Blood Vessel                606
Colon                       596
Prostate                    595
Stomach                     588
Head And Neck Region        518
Adipose Tissue              515
Ovary                       507
Thyroid Gland               504
Liver                       479
Bladder                     416
Muscle                      396
Heart                       377
Pancreas                    345
Blood                       337
Testis                      313
Cervix                      304
Thyroid                     279
Nerve                       278
Soft Tissue,Bone            258
Adrenal Gland               205
Endometrium                 180
Paraganglia                 177
Uterus                      135
Thymus                      119
Pituitary                   107
Spleen  

In [33]:
#save to csv
path = 'ATG7/dataframes_for_input/'
df_ATG7_NormPrim.to_csv(output_file_directory+path+'df_ATG7_NormPrim.csv',sep = ",", index=False)